In [1]:
import pandas as pd

In [26]:
week1 = pd.read_csv("Restaurant - Week 1 Sales.csv")
week2 = pd.read_csv("Restaurant - Week 2 Sales.csv")
customers = pd.read_csv("Restaurant - Customers.csv")
food = pd.read_csv("Restaurant - Foods.csv")
satis = pd.read_csv("Restaurant - Week 1 Satisfaction.csv")

In [3]:
# combine week 1 and week 2 sales
pd.concat(objs = [week1, week2])
# you can ignore previous indices using ignore_index

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


In [4]:
week1.append(week2)

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,783,10
246,556,10
247,547,9
248,252,9


We can use multiindex to keep track of where the data came from

In [5]:
sales = pd.concat(objs = [week1, week2], keys=["Week 1", "Week 2"])
sales

Customer ID  Food ID
Week 1 0            537        9
       1             97        4
       2            658        1
       3            202        2
       4            155        9
...                 ...      ...
Week 2 245          783       10
       246          556       10
       247          547        9
       248          252        9
       249          249        6

[500 rows x 2 columns]

In [6]:
sales.loc[("Week 1",)]

,Customer ID,Food ID
0,537,9
1,97,4
2,658,1
3,202,2
4,155,9
...,...,...
245,413,9
246,926,6
247,134,3
248,396,6


Merging (inner):

In [9]:
week1.merge(week2, on = "Customer ID", suffixes = ["-W1","-W2"]) # inner join by default
# number of occurences =  product of occurences (observe ID 155)
# pd.merge(week1, week2, .....) works as well

,Customer ID,Food ID-W1,Food ID-W2
0,537,9,5
1,155,9,3
2,155,1,3
3,503,5,8
4,503,5,9
...,...,...,...
57,945,5,4
58,343,3,5
59,343,3,2
60,343,3,7


In [11]:
# instances of customers buying the same food in both weeks
week1.merge(week2, on = ["Customer ID", "Food ID"]) 

,Customer ID,Food ID
0,304,3
1,540,3
2,937,10
3,233,3
4,21,4
5,21,4
6,922,1
7,578,5
8,578,5


Merge (outer):

In [14]:
week1.merge(week2, how = "outer", on = "Customer ID", suffixes = ["-W1","-W2"], indicator = True )

,Customer ID,Food ID-W1,Food ID-W2,_merge
0,537,9.0,5.0,both
1,97,4.0,NaN,left_only
2,658,1.0,NaN,left_only
3,202,2.0,NaN,left_only
4,155,9.0,3.0,both
...,...,...,...,...
449,855,NaN,4.0,right_only
450,559,NaN,10.0,right_only
451,276,NaN,4.0,right_only
452,556,NaN,10.0,right_only


In [18]:
week1.merge(week2, how = "outer", on = "Customer ID", suffixes = ["-W1","-W2"], indicator = True)["_merge"].value_counts()

right_only    197
left_only     195
both           62
Name: _merge, dtype: int64

Merge (left):

In [21]:
week1.merge(food, how = "left", on = "Food ID", sort = True)

,Customer ID,Food ID,Food Item,Price
0,658,1,Sushi,3.99
1,600,1,Sushi,3.99
2,155,1,Sushi,3.99
3,341,1,Sushi,3.99
4,20,1,Sushi,3.99
...,...,...,...,...
245,809,10,Drink,1.75
246,584,10,Drink,1.75
247,274,10,Drink,1.75
248,151,10,Drink,1.75


In [22]:
# when merging column names don't match:
week1.merge(customers, how = "left", left_on = "Customer ID", right_on = "ID")
# append .drop("ID", axis = 1) to remove redundancy in ID

,Customer ID,Food ID,ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,537,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,97,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,658,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,202,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,155,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...,...
245,413,9,413,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,926,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,134,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,396,Juan,Romero,Male,Zoonder,Analyst Programmer


In [28]:
# merging on index:
customers.set_index("ID", inplace = True)
food.set_index("Food ID", inplace = True)

In [33]:
week1sales = week1.merge(customers, how = "left", left_on = "Customer ID", right_index = True)
week1sales
# does not create duplicate columns

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III
...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer


In [35]:
week1sales = week1sales.merge(food, how = "left", left_on = "Food ID", right_index = True)
week1sales

,Customer ID,Food ID,First Name,Last Name,Gender,Company,Occupation,Food Item,Price
0,537,9,Cheryl,Carroll,Female,Zoombeat,Registered Nurse,Donut,0.99
1,97,4,Amanda,Watkins,Female,Ozu,Account Coordinator,Quesadilla,4.25
2,658,1,Patrick,Webb,Male,Browsebug,Community Outreach Specialist,Sushi,3.99
3,202,2,Louis,Campbell,Male,Rhynoodle,Account Representative III,Burrito,9.99
4,155,9,Carolyn,Diaz,Female,Gigazoom,Database Administrator III,Donut,0.99
...,...,...,...,...,...,...,...,...,...
245,413,9,Diane,Bailey,Female,Wikibox,Technical Writer,Donut,0.99
246,926,6,Anne,Wagner,Female,Skyba,Legal Assistant,Pasta,13.99
247,134,3,Diana,Hall,Female,Quinu,Financial Advisor,Taco,2.99
248,396,6,Juan,Romero,Male,Zoonder,Analyst Programmer,Pasta,13.99


In [36]:
week1.merge(week2, how = "left", left_index = True, right_index = True)

,Customer ID_x,Food ID_x,Customer ID_y,Food ID_y
0,537,9,688,10
1,97,4,813,7
2,658,1,495,10
3,202,2,189,5
4,155,9,267,3
...,...,...,...,...
245,413,9,783,10
246,926,6,556,10
247,134,3,547,9
248,396,6,252,9


Joining:

In [37]:
week1.join(satis)
# basically a left join

,Customer ID,Food ID,Satisfaction Rating
0,537,9,2
1,97,4,7
2,658,1,3
3,202,2,7
4,155,9,10
...,...,...,...
245,413,9,1
246,926,6,2
247,134,3,8
248,396,6,10
